# Tasks

What is a Julia Task?

- very lightweight coroutines
- Not threads!
- Internal to and scheduled by a Julia Process

In [1]:
?Task

search: Task task_local_storage @task istaskdone istaskstarted current_task



```
Task(func)
```

Create a `Task` (i.e. coroutine) to execute the given function (which must be callable with no arguments). The task exits when this function returns.


In [2]:
function mytask()
    println("Going to take a nap.")
    sleep(10)
    println("Woke up.")
    rand()
end

t=Task(mytask)

Task (runnable) @0x00007f9da84230d0

## scheduling a task

**`schedule`** actually starts the task, but will *return immediately*

In [3]:
schedule(t)

Going to take a nap.


Task (runnable) @0x00007f9da84230d0

## waiting on a task

In [4]:
println("Doing something else while t is taking a nap...")
inv(rand(100, 100))
@time @show wait(t)
@show t.state
println("task finished")

Doing something else while t is taking a nap...
Woke up.
wait(t) = 0.12803393161762378
  8.965076 seconds (33.73 k allocations: 1.516 MB)
t.state = :done
task finished


## @async - syntax sugar for creating tasks and scheduling it

In [5]:
t=@async begin
    println("Going to take a nap.")
    sleep(5)
    println("Woke up.")
end

Going to take a nap.


Task (runnable) @0x00007f9daa2409d0

In [6]:
21+21

42

## Channels

Allows communication between Tasks

In [7]:
input = Channel()
result = Channel()
doubler = @async while true
    x = take!(input)
    println("Got message")
    put!(result, 2x)
end

printer = @async while true
    res = take!(result)
    @show res
end

Task (runnable) @0x00007f9daa5da230

In [8]:
using Interact

In [9]:
@manipulate for i=1:100
    put!(input, i)
end

Interact.Options{:SelectionSlider,Int64}(Signal{Int64}(50, nactions=1),"i",50,"50",Interact.OptionDict(DataStructures.OrderedDict("1"=>1,"2"=>2,"3"=>3,"4"=>4,"5"=>5,"6"=>6,"7"=>7,"8"=>8,"9"=>9,"10"=>10…),Dict(68=>"68",2=>"2",89=>"89",11=>"11",39=>"39",46=>"46",85=>"85",25=>"25",55=>"55",42=>"42"…)),Any[],Any[],true,"horizontal")

Woke up.
Got message


50

## Adding Julia Processes, running "Remote Tasks"

In [10]:
# Run if on JuliaRun on ulam.astar
include("/home/juser/.machines.jl")
add_workers(4)

LoadError: could not open file /home/juser/.machines.jl

In [11]:
# Run if using the notebook on your own computer
addprocs(4)

4-element Array{Int64,1}:
 2
 3
 4
 5

In [12]:
procs()

5-element Array{Int64,1}:
 1
 2
 3
 4
 5

## Estimate pi in parallel

In [13]:
@everywhere function trials(numsteps=1000)  # default value of the parameter
    pos = 0 
    for j in 1:numsteps
        pos += Int(rand()^2 + rand()^2 < 1)
    end
    return pos
end

function estimate_pi(in_circle, N)
    4in_circle / N
end

estimate_pi (generic function with 1 method)

In [14]:
estimate_pi(trials(10^8), 10^8)

3.1416224

In [15]:
# @spawnat is like @async but runs on a different process
f=@spawnat 3 begin
    println("Process ", myid(), " starting random trials")
    res = trials(10^8)
    println("Process ", myid(), " done")
    res
end

Future(3,1,10,Nullable{Any}())

	From worker 3:	Process 3 starting random trials
	From worker 3:	Process 3 done


In [16]:
typeof(f)

Future

What's the curious `Future(3,1,12,Nullable{Any}())` thing?

In [17]:
f[]

78546544

In [18]:
# @spawnat is like @async but runs on a different process
function remote_trials(pid,n)
    @spawnat pid begin
        println("Process ", myid(), " starting trials")
        trials(n)
    end
end

remote_trials (generic function with 1 method)

In [19]:
remote_trials(2, 1000)

Future(2,1,12,Nullable{Any}())

	From worker 2:	Process 2 starting random walk


In [21]:
function parallel_trials(n, pids=workers())
    @time futures = [remote_trials(p,n) for p in pids]
    sum([f[] for f in futures])
end

parallel_trials (generic function with 2 methods)

In [22]:
@time estimate_pi(parallel_trials(10^8), 10^8*nworkers())

 	From worker 2:	Process 2 starting random walk
 0.017125 seconds (2.43 k allocations: 95.375 KB)
	From worker 3:	Process 3 starting random walk
	From worker 4:	Process 4 starting random walk
	From worker 5:	Process 5 starting random walk
  1.742937 seconds (51.63 k allocations: 2.473 MB)


3.14165907

In [23]:
@time estimate_pi(trials(10^8*nworkers()), 10^8*nworkers())

  2.324070 seconds (10 allocations: 256 bytes)


3.14155176

## Fun using TCP: The Julia Workshop Cluster

The following server is running on anubis.juliacomputing.com

It listens to connections from YOU and requests you to run a pi estimation simulation of 10^8 trials. After every user is done, it reads the result and updates the estimate.

```
### Server:

function estimate_pi(in_circle, N)
    4in_circle / N
end

function run_server(n=10^8)
    results = Channel()
    @async begin
        srvr = listen(8000)
        while true
            sock = accept(srvr)
            serialize(sock, n)
            @async begin
                put!(results, deserialize(sock))
            end
        end
    end

    running_tally = (0, 0)
    @async while true
        c, trials = take!(results)
        C, total_trials = running_tally
        running_tally = (C+c, trials+total_trials)
        println("Total samples: ", running_tally[2])
        println("Current estimate: ", estimate_pi(running_tally...))
    end
end
```

Run the below code to run the simulation and send it to the server

In [24]:
# Make available your local computation resources

@async begin
    c = connect("anubis.juliacomputing.com", 8000)
    n = deserialize(c)    # <--- Block wait for a request    
    serialize(c, (parallel_trials(n), nworkers()))
    close(c)
end


Task (runnable) @0x00007f9daa9cd600

	From worker 2:	Process 2 starting random walk
  0.008215 seconds (277 allocations: 14.828 KB)
	From worker 5:	Process 5 starting random walk
	From worker 3:	Process 3 starting random walk
	From worker 4:	Process 4 starting random walk
